In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
import numpy as np
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import glob

class BrainTumorDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.subject_dirs = sorted([os.path.join(data_dir, name) for name in os.listdir(data_dir)])
        # print(self.subject_dirs)

    def __len__(self):
        return len(self.subject_dirs)

    def __getitem__(self, idx):
        subject_dir = self.subject_dirs[idx]
        image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
        cropped_image_path = glob.glob(os.path.join(subject_dir, '*t1n-voided.nii.gz'))[0]
        healthy_mask_path = glob.glob(os.path.join(subject_dir, '*healthy.nii.gz'))[0]
        unhealthy_mask_path = glob.glob(os.path.join(subject_dir, '*unhealthy.nii.gz'))[0]


        # 加载图像和掩膜
        image = nib.load(image_path).get_fdata()
        healthy_mask = nib.load(healthy_mask_path).get_fdata()
        unhealthy_mask = nib.load(unhealthy_mask_path).get_fdata()
        cropped_image = nib.load(cropped_image_path).get_fdata()
        mask_affine = nib.load(healthy_mask_path).affine
        image = np.pad(image, ((0, 0), (0, 0), (3, 2)), mode='constant')
        healthy_mask = np.pad(healthy_mask, ((0, 0), (0, 0), (3, 2)), mode='constant')
        cropped_image = np.pad(cropped_image, ((0, 0), (0, 0), (3, 2)), mode='constant')
        unhealthy_mask = np.pad(unhealthy_mask, ((0, 0), (0, 0), (3, 2)), mode='constant')
        # print(mask_affine)


        # 二值化掩膜
        unhealthy_mask = np.where(unhealthy_mask > 0, 1, 0)
        unhealthy_mask_bool = (unhealthy_mask == 1)

        # 使用掩码对图像进行裁剪
        image[unhealthy_mask_bool] = 0

        image = image[np.newaxis, ...]
        cropped_image = cropped_image[np.newaxis, ...]
        healthy_mask = healthy_mask[np.newaxis, ...]

        # image 是健康的图像，即只抠掉肿瘤区域的图像 cropped_image是抠掉要生成区域的图像，mask是健康图像的掩膜
        return image, cropped_image, healthy_mask, mask_affine


In [6]:
import os
import numpy as np
import nibabel as nib
import glob
# 指定数据路径
data_dir = 'D:\\BraTS\\BraTS-2023_challenge\\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training'
subject_dirs = sorted([os.path.join(data_dir, name) for name in os.listdir(data_dir)])
# 初始化一个大的四维数组来存储所有的patch
all_patches = np.zeros((len(subject_dirs)*16, 120, 120, 40))
temp = 0
# 遍历每个子文件夹
for idx in range(len(subject_dirs)):
    subject_path = os.path.join(data_dir, subject_dir)
    
    # 读取nii.gz文件
    subject_dir = subject_dirs[idx]
    image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
    print(image_path)
    image = nib.load(image_path).get_fdata()
    image = np.pad(image, ((0, 0), (0, 0), (3, 2)), mode='constant')
    # temp = 0
    # 将三维数据分成16个patch
    for i in range(2):
        for j in range(2):
            for k in range(4):
                temp += 1
                patch = image[i*120:(i+1)*120, j*120:(j+1)*120, k*40:(k+1)*40]
                all_patches[temp-1, :, :, :] = patch

# 保存所有的patch到一个大的四维数组中
np.save('all_patches.npy', all_patches)

D:\BraTS\BraTS-2023_challenge\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00000-000\BraTS-GLI-00000-000-t1n.nii.gz
D:\BraTS\BraTS-2023_challenge\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00002-000\BraTS-GLI-00002-000-t1n.nii.gz
D:\BraTS\BraTS-2023_challenge\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00003-000\BraTS-GLI-00003-000-t1n.nii.gz
D:\BraTS\BraTS-2023_challenge\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00005-000\BraTS-GLI-00005-000-t1n.nii.gz
D:\BraTS\BraTS-2023_challenge\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00006-000\BraTS-GLI-00006-000-t1n.nii.gz
D:\BraTS\BraTS-2023_challenge\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00008-000\BraTS-GLI-00008-000-t1n.nii.gz
D:\BraTS\BraTS-2023_challenge\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00008-001\BraTS-GLI-00008-001-t1n.nii.gz
D:\BraTS\BraTS-2023_challenge\ASNR